In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


def computeCatenary(x1,y1,x2,y2,s_user):
    p1 = [x1,y1]
    p2 = [x2,y2]

    v = np.abs(p1[1] - p2[1])
    h = np.abs(p1[0] - p2[0])
    s = np.sqrt(v*v + h*h) + s_user #minimum length + user defined length

    def ctn(a,x):
        return a*np.cosh(x/a)

    def f(a):
        return np.sqrt(s*s - v*v) - 2*a*np.sinh(h/(2*a))

    def f_sq(a):
        return f(a)*f(a)

    res = minimize(lambda a: f_sq(a), x0=1)
    a = res.x[0]


    #find lowest point in global frame, this idea from https://mysite.du.edu/~jcalvert/math/catenary.htm
    def C(x, x1, y1, x2, y2, a):
        xm = (x1+x2)/2
        deltax = (x2-x1)/2
        return (y2-y1)/a - 2*np.sinh((xm-x)/a)*np.sinh(deltax/a) 

    def C_sq(x):
        return C(x, p1[0], p1[1], p2[0], p2[1], a) * C(x, p1[0], p1[1], p2[0], p2[1], a)

    res2 = minimize(lambda x: C_sq(x), x0=1)
    xlow = res2.x[0] # this is the x coordinate of the lowest point in the catenary (marked by the vertical line)


    #in a local reference frame, compute the catenary as if the lowest point ylow is at x=0
    xlocal = np.arange(p1[0]-xlow, p2[0]-xlow, 0.01)
    ylocal = a*np.cosh(xlocal/a)
    ytransform = p1[1] - ylocal[0]
    xtransform = xlow

    plt.figure(figsize=[20,10])
    plt.scatter(p1[0], p1[1], color='k')
    plt.scatter(p2[0], p2[1], color='k')
    plt.axvline(xlow, linestyle='--')

    plt.plot(xlocal + xtransform, ylocal + ytransform, color='y')
    plt.xlim([0, 400])
    plt.ylim([-200,200])
    return h,v,s,a,xlow

interact(computeCatenary, x1=widgets.IntSlider(min=1,max=100,step=1,value=20),
                          y1=widgets.IntSlider(min=-200,max=200,step=1,value=50),
                          x2=widgets.IntSlider(min=1,max=300,step=1,value=120), 
                          y2=widgets.IntSlider(min=-200,max=200,step=1,value=4),
                          s_user=widgets.IntSlider(min=1,max=400,step=1,value=100));

interactive(children=(IntSlider(value=20, description='x1', min=1), IntSlider(value=50, description='y1', max=200, min=-200), IntSlider(value=120, description='x2', max=300, min=1), IntSlider(value=4, description='y2', max=200, min=-200), IntSlider(value=100, description='s_user', max=400, min=1), Output()), _dom_classes=('widget-interact',))